In [1]:
!pip install einops

In [2]:
!pip install opendatasets



In [3]:
!pip install pandas

In [4]:
import opendatasets as od
import pandas
#{"username":"pari16","key":"dfd22a0d64f82186cd7db7c1a9af2044"}
od.download("https://www.kaggle.com/datasets/preatcher/standard-ocr-dataset")

Skipping, found downloaded files in "./standard-ocr-dataset" (use force=True to force download)


In [5]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
import numpy as np
from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary

In [6]:
training_data_path="/content/standard-ocr-dataset/data/training_data"
training_data=[]
import os
for label in os.listdir(training_data_path):#os.listdir(training_data_path) gives list of list of 0 images,1 images etc =[0,1,2,3,4,5,6,7,8,9,'A','B',....,'Z']
  for j in os.listdir(os.path.join(training_data_path,label)):
    training_data.append([os.path.join(training_data_path,label,j),label])


In [7]:
LABELS = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'A':10,'B':11,'C':12,'D':13,'E':14,'F':15,'G':16,'H':17,'I':18,'J':19,'K':20,'L':21,'M':22,'N':23,'O':24,'P':25,'Q':26,'R':27,'S':28,'T':29,'U':30,'V':31,'W':32,'X':33,'Y':34,'Z':35}

In [8]:
# resize to imagenet size
transform = Compose([Resize((224, 224)), ToTensor()])


In [9]:
from torch.utils.data import Dataset,DataLoader
class Train_Dataset(Dataset):
  def __init__(self,img_list):
    self.img_list=img_list
  def __getitem__(self,index):
    path=self.img_list[index][0]
    label=self.img_list[index][1]
    one_hot= np.eye(36)[LABELS[label]]
    img=Image.open(path)
    #img=transform(img)
    x=transform(img)
    #x = x.unsqueeze(0)
    return [x,one_hot]     #return[img,label]
  def __len__(self):
    return len(self.img_list)


In [10]:
train_dataloader=DataLoader(Train_Dataset(training_data))

In [11]:
for i in train_dataloader:
  print(i[1])
  break

tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       dtype=torch.float64)


In [12]:
patch_size = 16            # 16 pixels
for i in train_dataloader:
  pathes = rearrange(i[0], 'b c (h s1) (w s2) -> b (h w) (s1 s2 c)', s1=patch_size, s2=patch_size)

In [13]:
testing_data_path="/content/standard-ocr-dataset/data/testing_data"
testing_data=[]
import os
for label in os.listdir(testing_data_path):#os.listdir(testing_data_path) gives list of list of 0 images,1 images etc =[0,1,2,3,4,5,6,7,8,9,'A','B',....,'Z']
  for j in os.listdir(os.path.join(testing_data_path,label)):
    testing_data.append([os.path.join(testing_data_path,label,j),label])

In [14]:
class Test_Dataset(Dataset):
  def __init__(self,img_list):
    self.img_list=img_list
  def __getitem__(self,index):
    path=self.img_list[index][0]
    label=self.img_list[index][1]
    one_hot= np.eye(36)[LABELS[label]]
    img=Image.open(path)
    #img=transform(img)
    x=transform(img)
    #x = x.unsqueeze(0)
    return [x,one_hot]     #return[img,label]
  def __len__(self):
    return len(self.img_list)

In [15]:
test_dataloader=DataLoader(Test_Dataset(testing_data))

In [16]:
for i in test_dataloader:
  print(i[1])
  break

tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       dtype=torch.float64)


In [17]:
'''for i in train_dataloader:
  print(i[0].shape)            #torch.Size([1,1, 224, 224])
  break'''

'for i in train_dataloader:\n  print(i[0].shape)            #torch.Size([1,1, 224, 224])\n  break'

In [18]:
patch_size = 16            # 16 pixels
for i in test_dataloader:
  pathes = rearrange(i[0], 'b c (h s1) (w s2) -> b (h w) (s1 s2 c)', s1=patch_size, s2=patch_size)




In [19]:
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels= 1, patch_size= 16, emb_size= 128, img_size= 224):
        self.patch_size = patch_size
        super().__init__()
        '''self.projection = nn.Sequential(
            # break-down the image in s1 x s2 patches and flat them
            Rearrange('b c (h s1) (w s2) -> b (h w) (s1 s2 c)', s1=patch_size, s2=patch_size),         #size is 196X256
            nn.Linear(patch_size * patch_size * in_channels, emb_size)                                 # size is 768X128
        )'''
        self.projection = nn.Sequential(
            # using a conv layer instead of a linear one -> performance gains
            nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size),
            Rearrange('b e (h) (w) -> b (h w) e'),
        )
        self.cls_token = nn.Parameter(torch.randn(1,1, emb_size))
        self.positions = nn.Parameter(torch.randn((img_size // patch_size) **2 + 1, emb_size))


    def forward(self, x: Tensor) -> Tensor:
        b, _, _, _ = x.shape
        #print("Input shape:", x.shape)
        x = self.projection(x)
        #print("Intermediate shape:", x.shape)
        cls_tokens = repeat(self.cls_token, '() n e -> b n e', b=b)
        # prepend the cls token to the input
        x = torch.cat([cls_tokens, x], dim=1)
        # add position embedding
        x += self.positions
        return x


for i in train_dataloader:
  print(PatchEmbedding()(i[0]).shape)
  break

torch.Size([1, 197, 128])


In [20]:
class MultiHeadAttention(nn.Module):
    def __init__(self, emb_size = 128, num_heads = 8, dropout = 0):
        super().__init__()
        self.emb_size = emb_size
        self.num_heads = num_heads
        # fuse the queries, keys and values in one matrix
        self.qkv = nn.Linear(emb_size, emb_size * 3)
        self.att_drop = nn.Dropout(dropout)
        self.projection = nn.Linear(emb_size, emb_size)

    def forward(self, x : Tensor, mask: Tensor = None) -> Tensor:
        # split keys, queries and values in num_heads
        qkv = rearrange(self.qkv(x), "b n (h d qkv) -> (qkv) b h n d", h=self.num_heads, qkv=3)
        queries, keys, values = qkv[0], qkv[1], qkv[2]
        # sum up over the last axis
        energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys) # batch, num_heads, query_len, key_len
        if mask is not None:
            fill_value = torch.finfo(torch.float32).min
            energy.mask_fill(~mask, fill_value)

        scaling = self.emb_size ** (1/2)
        att = F.softmax(energy, dim=-1) / scaling
        att = self.att_drop(att)
        # sum up over the third axis
        out = torch.einsum('bhal, bhlv -> bhav ', att, values)
        out = rearrange(out, "b h n d -> b n (h d)")
        out = self.projection(out)
        return out



"the embedding size refers to the number of features or dimensions used to represent each item in a dataset"

In [21]:
class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):     #**kwargs, which is a special syntax that allows passing additional keyword arguments if needed.
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x


In [22]:
class FeedForwardBlock(nn.Sequential):
    def __init__(self, emb_size, expansion= 4, drop_p= 0.):
        super().__init__(
            nn.Linear(emb_size, expansion * emb_size),
            nn.GELU(),
            nn.Dropout(drop_p),
            nn.Linear(expansion * emb_size, emb_size),
        )


In [23]:
class TransformerEncoderBlock(nn.Sequential):
    def __init__(self,
                 emb_size: int = 128,
                 drop_p: float = 0.,
                 forward_expansion: int = 4,
                 forward_drop_p: float = 0.,
                 ** kwargs):
        super().__init__(
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                MultiHeadAttention(emb_size, **kwargs),
                nn.Dropout(drop_p)
            )),
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                FeedForwardBlock(
                    emb_size, expansion=forward_expansion, drop_p=forward_drop_p),
                nn.Dropout(drop_p)
            )
            ))


In [24]:
class TransformerEncoder(nn.Sequential):
    def __init__(self, depth: int = 12, **kwargs):
        super().__init__(*[TransformerEncoderBlock(**kwargs) for _ in range(depth)])


In [25]:
class ClassificationHead(nn.Sequential):
    def __init__(self, emb_size = 128, n_classes =36):
        super().__init__(
            Reduce('b n e -> b e', reduction='mean'),
            nn.LayerNorm(emb_size),
            nn.Linear(emb_size, n_classes))


In [26]:
from tqdm import tqdm
import torch.optim as optim

In [37]:
class ViT(nn.Sequential):
    def __init__(self,
                in_channels= 1,
                patch_size= 16,
                emb_size= 128,
                img_size= 224,
                depth= 12,
                n_classes= 36,
                **kwargs):
        super().__init__(
            PatchEmbedding(in_channels, patch_size, emb_size, img_size),
            TransformerEncoder(depth, emb_size=emb_size, **kwargs),
            ClassificationHead(emb_size, n_classes)
        )


    #train_dataloader=train_dataloader.type(torch.LongTensor)

    def train(self,EPOCHS=10):

      optimizer = optim.Adam(self.parameters(), lr=0.001)
      #loss_function=nn.MSELoss()
      loss_function = nn.CrossEntropyLoss()
      self.EPOCHS=EPOCHS

      '''for epoch in range(self.EPOCHS):
        for batch_X, batch_y in train_dataloader:  # Unpack the batch data and labels properly
            optimizer.zero_grad()
            outputs = self.forward(batch_X)
            loss = loss_function(outputs, batch_y)  # Use batch_y directly as the target
            loss.backward()
            optimizer.step()
        print(loss)'''
      for epoch in range(self.EPOCHS):
        for i in train_dataloader:
            batch_X,batch_y=i

            optimizer.zero_grad()
            outputs = self.forward(batch_X)
           # batch_y = batch_y.view(-1)
            #outputs=outputs.t()
           #print(outputs.shape)
           # print(batch_y.shape)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()
        print(loss)







    def test(self):
      correct = 0
      total = len(test_dataloader)
     # with torch.no_grad():
      for i in test_dataloader:
        test_X,test_y=i
        output=self.forward(test_X)

        for j in range(len(test_y)):
          real_class = torch.argmax(test_y,1)[j]
          predicted_class = torch.argmax(output,1)[j]
        if predicted_class == real_class:
            correct += 1

      print("Accuracy:", round(correct/total*100,2))


In [45]:
vit=ViT().cuda()

In [ ]:
vit.train()

In [ ]:
vit.test()

In [46]:
summary(vit,(1,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 14, 14]          32,896
         Rearrange-2             [-1, 196, 128]               0
    PatchEmbedding-3             [-1, 197, 128]               0
         LayerNorm-4             [-1, 197, 128]             256
            Linear-5             [-1, 197, 384]          49,536
           Dropout-6          [-1, 8, 197, 197]               0
            Linear-7             [-1, 197, 128]          16,512
MultiHeadAttention-8             [-1, 197, 128]               0
           Dropout-9             [-1, 197, 128]               0
      ResidualAdd-10             [-1, 197, 128]               0
        LayerNorm-11             [-1, 197, 128]             256
           Linear-12             [-1, 197, 512]          66,048
             GELU-13             [-1, 197, 512]               0
          Dropout-14             [-1, 1